In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from astropy.convolution import AiryDisk2DKernel
import scipy.signal
%matplotlib inline
#plt.style.use('seaborn-colorblind')

In [ ]:
img = np.array(Image.open('bike2.png'))
plt.imshow(img,cmap='gray')
plt.axis('off')
plt.savefig("bike.png", bbox_inches='tight')

In [ ]:
airydisk_2D_kernel = AiryDisk2DKernel(10)

In [ ]:
plt.imshow(airydisk_2D_kernel,cmap='gray_r')
plt.axis('off')
plt.savefig("airy.png", bbox_inches='tight')

In [ ]:
convolved = scipy.signal.convolve2d(255.0-img,np.array(airydisk_2D_kernel),'same')
plt.imshow(convolved,cmap='gray_r')
plt.axis('off')
plt.savefig("convolved.png", bbox_inches='tight')

In [ ]:
import random
prob = 0.01
cnt = 0
img_out = np.zeros_like(img)
for i in range(1000):
    #plt.figure()
    img2 = np.zeros_like(img)
    for y in xrange(img2.shape[0]):
        for x in xrange(img2.shape[1]):
            if img[y,x] == 0:
                if random.random() < prob:
                    img2[y,x] = 255.0
    convolved = scipy.signal.convolve2d(img2,np.array(airydisk_2D_kernel),'same')
    if convolved.max() > 3.0 and convolved.max() < 6.0:
        res = np.where(convolved == convolved.max())
        cnt+=1
        for yo,xo in zip(res[0],res[1]):
            img_out[yo,xo] += 1.0
print cnt
plt.imshow(img_out,cmap='gray_r')
plt.axis('off')
plt.savefig("res.png" , bbox_inches='tight')

In [ ]:
import random
from sklearn.linear_model import Lasso

In [ ]:
A = np.zeros((img.ravel().shape[0],img.ravel().shape[0]))
for i in xrange(img.ravel().shape[0]):
    img2 = np.zeros_like(img)
    img2[int(i)/int(img2.shape[0]),i%img2.shape[0]] = 1
    convolved = scipy.signal.convolve2d(img2,np.array(airydisk_2D_kernel),'same')
    A[i,:] = convolved.ravel()

In [ ]:
prob = 0.04
img_out = np.zeros_like(img)

for i in range(50):
    #plt.figure()
    img2 = np.zeros_like(img)
    for y in xrange(img2.shape[0]):
        for x in xrange(img2.shape[1]):
            if img[y,x] == 0:
                if random.random() < prob:
                    img2[y,x] = 255.0
    convolved = scipy.signal.convolve2d(img2,np.array(airydisk_2D_kernel),'same')
    y = convolved.ravel()
    clf = Lasso(alpha=0.00003,fit_intercept=False)
    clf.fit(A,y)
    #print A.shape,y.shape
    #print clf.coef_.sum()
    img_out += clf.coef_.reshape(img2.shape).astype(np.uint8)
        
plt.imshow(img_out,cmap='gray_r')
plt.axis('off')
plt.savefig("sparse.png" , bbox_inches='tight')

In [ ]:
plt.imshow(convolved,cmap='gray_r')
plt.axis('off')
plt.savefig("sparse.png" , bbox_inches='tight')